In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
from pathlib import Path
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


current_file = Path.cwd()
parent_directory = current_file.parent

In [ ]:
# Base URL without the pagination parameter
transaction_type = 'venta'
property_type = 'departamento'
listing_category = 'propiedades-usadas'
region_name = 'metropolitana'
base_url = f"https://www.portalinmobiliario.com/{transaction_type}/{property_type}/{listing_category}/{region_name}/"

start = 1  # Starting point for pagination
increment = 48  # Step size for pagination
max_pages = 500  # Set a limit to avoid infinite loops

# List to store all extracted links
all_links = []

def fetch_with_rate_limit(url, max_retries=3):
    """Fetches a URL with retry and rate limiting."""
    for _ in range(max_retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response
            elif response.status_code in (429, 503):  # Too Many Requests or Service Unavailable
                retry_after = response.headers.get("Retry-After")
                retry_after = int(retry_after) if retry_after and retry_after.isdigit() else random.uniform(1, 3)
                print(f"Rate limited. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
            else:
                print(f"Unexpected status: {response.status_code}. Retrying...")
                time.sleep(random.uniform(1, 5))
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(random.uniform(1, 3))
    return None

def scrape_page(url):
    """Scrapes a single page and returns links."""
    response = fetch_with_rate_limit(url)
    if response:
        soup = BeautifulSoup(response.content, "html.parser")
        list_items = soup.select("main > div > div:nth-of-type(3) > section > ol > li")
        links = [
            item.find("a", class_="ui-search-result__image ui-search-link")['href']
            for item in list_items if item.find("a", class_="ui-search-result__image ui-search-link")
        ]
        return links
    return []

# Main scraping loop
for _ in range(max_pages):
    current_url = f"{base_url}_Desde_{start}_OrderId_PRICE_NoIndex_True"
    links = scrape_page(current_url)

    if not links:  # Stop if no links are found
        print("No more items found. Ending scrape.")
        break

    all_links.extend(links)
    print(f"Scraped {len(links)} items from page starting at {start}.")
    start += increment
    time.sleep(random.uniform(2, 5))  # Random delay to reduce server load

cleaned_links = [
    re.search(r'(https://www\.portalinmobiliario\.com/MLC-\d+)', url).group(1)
    for url in all_links if re.search(r'(https://www\.portalinmobiliario\.com/MLC-\d+)', url)
]

# Create DataFrame and save results
links_df = pd.DataFrame({
    "transaction_type": transaction_type,
    "property_type": property_type,
    "listing_category": listing_category,
    "region_name": region_name,
    "url": cleaned_links
})

output_file = parent_directory / "data" / "raw" / 'scraped_links_portal_inmob.csv'
links_df.to_csv(output_file, index=False)
print("CSV file saved")

print(f"Total Links Collected: {len(all_links)}")

In [11]:
current_file = Path.cwd()
parent_directory = current_file.parent
links_path = parent_directory / "data" / "raw" / 'scraped_links_portal_inmob.csv'
links_df = pd.read_csv(links_path)
try:
    scraped_path = parent_directory / "data" / "raw" / 'scraped_apartments_portal_inmob.csv'
    scraped_df = pd.read_csv(scraped_path)
    # Narrow down links to scrape for apartments which haven't been scraped before
    remaining_links_df = links_df[~links_df['url'].isin(scraped_df['url'])].reset_index(drop=True)
except Exception as e:
    print("no scraped_df csv")
    print(f"error: {str(e)}")

# # FIX: Add apartments with no currency unit
# no_curr_scraped_links_df = scraped_df[scraped_df['currency'].isna()][['url']].copy()
# scraped_df = scraped_df[~scraped_df['currency'].isna()]
# no_curr_scraped_links_df['transaction_type'] = transaction_type
# no_curr_scraped_links_df['property_type'] = property_type
# no_curr_scraped_links_df['listing_category'] = listing_category
# no_curr_scraped_links_df['region_name'] = region_name
# cols_in_order = ['transaction_type', 'property_type', 'listing_category', 'region_name', 'url']
# no_curr_scraped_links_df = no_curr_scraped_links_df[cols_in_order]
# remaining_links_df = pd.concat([remaining_links_df, no_curr_scraped_links_df], ignore_index=True)
# remaining_links_df.drop_duplicates(inplace=True)

In [9]:
options = Options()
options.add_argument("--headless")

# Initialize an empty list to store extracted data
scraped_data = []

# Iterate over each URL in the DataFrame
batch_size = 10
driver = None
for index, row in remaining_links_df.iterrows():
    url = row['url']
    print(f"Scraping: {url}")
    # Reinitialize the driver every 'batch_size' iterations
    if index % batch_size == 0:
        print("Entro")
        if driver is not None:
            driver.quit()  # Quit the old driver if it exists
        driver = webdriver.Chrome(options=options)
    try:
        # Load the page using Selenium to bypass anti-bot measures
        driver.get(url)
        # Must refresh to bypass anti-bot measures
        driver.refresh()
        time.sleep(random.uniform(0, 1))
        WebDriverWait(driver, 3).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "ui-vpp-striped-specs__table"))
        )
        # Get the rendered page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract data using the provided selectors
        title = soup.select_one("#header > div > div.ui-pdp-header__title-container > h1")
        subtitle = soup.select_one("#header > div > div.ui-pdp-header__subtitle > span")
        price = soup.select_one("#price > div > div > div > span > span > span.andes-money-amount__fraction")
        currency = soup.select_one("#price > div > div > div > span > span > span.andes-money-amount__currency-symbol")
        common_expenses = soup.select_one("#maintenance_fee_vis > p")
        squared_meters = soup.select_one("#highlighted_specs_res > div > div:nth-child(1) > span")
        dorms = soup.select_one("#highlighted_specs_res > div > div:nth-child(2) > span")
        bathrooms = soup.select_one("#highlighted_specs_res > div > div:nth-child(3) > span")
        location = soup.select_one("#location > div > div.ui-pdp-media.ui-vip-location__subtitle.ui-pdp-color--BLACK > div > p")

        # Extract coordinates from the map image srcset
        map_img = soup.select_one("#ui-vip-location__map > div > img")
        coordinates = None
        if map_img and 'srcset' in map_img.attrs:
            srcset = map_img['srcset']
            if "center=" in srcset:
                coordinates = srcset.split("center=")[1].split("&")[0]  # Extract lat,lng
                coordinates = coordinates.replace("%2C", ",")

        # Extract tables
        tables = soup.find_all("tbody", class_="andes-table__body")

        flat_table_data = {}
        for table in tables:
            rows = table.find_all("tr", class_="andes-table__row ui-vpp-striped-specs__row")
            for row in rows:
                header = row.find("th", class_="andes-table__header").text.strip() if row.find("th", class_="andes-table__header") else None
                value = row.find("td", class_="andes-table__column").text.strip() if row.find("td", class_="andes-table__column") else None
                if header and value:
                    flat_table_data[header] = value

        # Extract description
        description = soup.select_one("#description > div > div > div > p")

        # Extract verified seller info
        verified_seller = soup.select_one("#header > div > div.ui-pdp-seller-validated > p > a")

        # Extract image URL
        image = soup.select_one("#gallery > div > div > span:nth-child(3) > figure > img")
        image_url = image['src'] if image else None

        # Append the extracted data to the list
        data_entry = {
            "url": url,
            "title": title.text.strip() if title else None,
            "subtitle": subtitle.text.strip() if subtitle else None,
            "price": int(price.text.strip().replace(".", "")) if price else None,
            "currency": currency.text.strip() if currency else None,
            "common_expenses": common_expenses.text.strip() if common_expenses else None,
            "squared_meters": squared_meters.text.strip() if squared_meters else None,
            "dorms": dorms.text.strip() if dorms else None,
            "bathrooms": bathrooms.text.strip() if bathrooms else None,
            "location": location.text.strip() if location else None,
            "coordinates": coordinates,
            "description": description.text.strip() if description else None,
            "verified_seller": verified_seller.text.strip() if verified_seller else None,
            "image_url": image_url
        }

        data_entry.update(flat_table_data)
        scraped_data.append(data_entry)

    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Quit the Selenium driver
driver.quit()

Scraping: https://www.portalinmobiliario.com/MLC-2726723992
Entro
Scraping: https://www.portalinmobiliario.com/MLC-2772756714
Scraping: https://www.portalinmobiliario.com/MLC-2777065320
Scraping: https://www.portalinmobiliario.com/MLC-2778446752
Scraping: https://www.portalinmobiliario.com/MLC-2778554378
Scraping: https://www.portalinmobiliario.com/MLC-1544243983
Scraping: https://www.portalinmobiliario.com/MLC-1537685835
Scraping: https://www.portalinmobiliario.com/MLC-2679102934
Scraping: https://www.portalinmobiliario.com/MLC-2786159288
Scraping: https://www.portalinmobiliario.com/MLC-1553723949
Scraping: https://www.portalinmobiliario.com/MLC-2770777530
Entro
Scraping: https://www.portalinmobiliario.com/MLC-2789092370
Scraping: https://www.portalinmobiliario.com/MLC-1550859743
Scraping: https://www.portalinmobiliario.com/MLC-1551538641
Scraping: https://www.portalinmobiliario.com/MLC-1511600233
Scraping: https://www.portalinmobiliario.com/MLC-2794613248


In [10]:
# Check if `scraped_df` exists and update/append data accordingly
try:
    # If `scraped_df` already exists, append new data
    new_data_df = pd.DataFrame(scraped_data)
    scraped_df = pd.concat([scraped_df, new_data_df], ignore_index=True).drop_duplicates(subset="url")
except NameError:
    # If `scraped_df` doesn't exist, create it from `scraped_data`
    scraped_df = pd.DataFrame(scraped_data)

# Save the scraped data to a CSV file
output_file = parent_directory / "data" / "raw" / 'scraped_apartments_portal_inmob.csv'
scraped_df.to_csv(output_file, index=False)
print(f"Scraping completed and data saved to {output_file}.")

Scraping completed and data saved to /Users/felipemediavillalevinson/Documents/Hawspred/data/raw/scraped_apartments_portal_inmob.csv.


In [12]:
# Quit any remaining chrome instances
import os
import signal
import psutil

def kill_chrome_instances():
    for process in psutil.process_iter(attrs=['pid', 'name']):
        # Look for Chrome or chromedriver processes
        if 'chrome' in process.info['name'].lower() or 'chromedriver' in process.info['name'].lower():
            try:
                os.kill(process.info['pid'], signal.SIGTERM)
                print(f"Terminated process {process.info['name']} (PID: {process.info['pid']})")
            except Exception as e:
                print(f"Could not terminate process {process.info['name']} (PID: {process.info['pid']}): {e}")

# Call the cleanup function
kill_chrome_instances()